In [ ]:
# Hybrid Recommender System with Sentiment Analysis (BERT-uncased)
pip install pandas tqdm numpy nltk scikit-learn transformers torch plotly
pip install tensorflow

# Step 1: Import Libraries

import pandas as pd
from tqdm import tqdm
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from transformers import BertTokenizer,BertModel,BertForSequenceClassification
import tensorflow
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go

# Check if a GPU is available
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model and move it to the device (GPU or CPU)
model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to preprocess text with BERT
def preprocess_with_bert(text, max_length=512):
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'][0].to(device)
    attention_mask = encoding['attention_mask'][0].to(device)
    return input_ids, attention_mask

# Function to get BERT embeddings
def bert_embedding(text):
    input_ids, attention_mask = preprocess_with_bert(text)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_states = outputs.last_hidden_state
    sentence_embedding = torch.mean(last_hidden_states, dim=1).cpu().numpy()
    return sentence_embedding.flatten()

# Example usage
sample_text = "This is a sample text for embedding."
embedding = bert_embedding(sample_text)
print(embedding)
